# Intro to Deep Learning
<table align="left"><td>
  <a target="_blank"  href="https://github.com/Clemson-AI/Intro/blob/master/Intro_to_Deep_Learning_CAI.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/Clemson-AI/Intro/blob/master/Intro_to_Deep_Learning_CAI.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# Tensors
![Tensor](https://miro.medium.com/max/1050/0*jGB1CGQ9HdeUwlgB)

In [43]:
# Rank 0
torch.tensor(.6)

In [ ]:
# Rank 1 
torch.tensor([.7, 1.4, 2.1])

In [56]:
# Rank 2
torch.randn(2,2)

tensor([[-0.5733, -0.4501],
        [ 0.7236,  0.9845]])

In [57]:
# Rank 4
torch.randn(2,2,2,2)

tensor([[[[-0.3028, -0.4098],
          [ 0.4015,  0.2455]],

         [[ 1.6076,  0.5838],
          [ 0.6200, -0.6811]]],


        [[[ 0.3436,  1.0570],
          [ 0.3909,  1.9183]],

         [[ 0.3407,  0.4702],
          [-0.7572, -0.5975]]]])

# Activation Function

\begin{equation}

Sigmoid(x) = \sigma(x) = \dfrac{1}{1 + \exp(-x)}

\end{equation}

​	
![Sigmoid](https://pytorch.org/docs/stable/_images/Sigmoid.png)
\begin{equation}
ReLU(x) = x^+ = max(0,x)
\end{equation}

![ReLu](https://pytorch.org/docs/stable/_images/ReLU.png)


Sigmoid

In [ ]:
# Rank 0
r0data = torch.tensor(.6)
# Rank 1 
r1data = torch.tensor([.7, 1.4, 2.1])

In [4]:
sig = nn.Sigmoid()

In [49]:
sig(r0data)

tensor(0.6457)

In [50]:
sig(r1data)

tensor([0.6682, 0.8022, 0.8909])

ReLU

In [51]:
relu = nn.ReLU()

In [7]:
relu(r1data)

tensor([0.7000, 1.4000, 2.1000])

In [52]:
relu(data)

tensor(0.5000)

# Loss
\begin{equation}
loss = \dfrac{\sum_{i=1}^{n}∣y_i−x_i∣}{n}
\end{equation}

In [53]:
data = torch.tensor([5.0,5.0,5.0])
truth = torch.tensor([7.0,8.0,9.0])

In [54]:
# mean absolute error (MAE)
loss = nn.L1Loss(reduction='mean')

In [55]:
loss(data, truth)

tensor(3.)

# Optimizer